In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os
import warnings
import glob

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 6)
pd.set_option("display.max_rows", 6)
np.random.seed(2)

2025-04-16 13:55:26.873186: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load the data and process it

In [15]:
"""
As in the tutorial, the provided dataset consists of two files for each traffic scene:

<scene_id>.edges
two columns containing node IDs
target, source
Note: The tutorial models directed edges with source -> target.
You can either use undirected edges by changing the implementation or adding the missing entries to the edges file,
e.g., to the line target, source, you add the line source target. If you want to be more fancy, you could also try to infer
which other pedestrians the source node can see in their field of view and only add those (this would model that the movement
decisions are based only on the pedestrians in the field of view.)
<scene_id>.nodes
seven columns with node properties and target values, which should be predicted 
node id, current x, current y, previous x, previous y, future x, future y
the previous x and y represents the location of the pedestrian 1 second ago (you can use those values directly or infer the
movement direction and some speed estimate yourself)
the future x and y represents the target value, i.e., the location where the pedestrian will be in 1 second
Note: Some pedestrians do not have a future x and y coordinate, so you need to filter those for prediction. However, you can
still use their current and previous location when predicting the future location of other pedestrians.
"""
"""
Sample data: 
file: dataset/13528908058.edges:
contains data: target, source
19585800, 19590700
19585800, 19595200
19585800, 20000100
19590700, 19595200
19590700, 20000100
19591900, 19594200
19591900, 19595300
19591900, 19595800
19592201, 19595800
19592400, 20000200
19592800, 20000200
19592800, 20000300
19594200, 19595300
19594200, 19595800
19595200, 20000100
19595300, 19595800
20000200, 20000300
19502500, -1

corresponding nodes file: 
file: dataset/13528908058.nodes
contains data: node id, current x, current y, previous x, previous y, future x, future y
19502500,40050.0,-16544.0,40176.0,-16619.0,40205.0,-16357.0
19585800,16802.0,-11108.0,16140.0,-10573.0,17831.0,-11792.0
19590700,16846.0,-10526.0,16079.0,-9694.0,17528.0,-11131.0
19591900,11346.0,-6253.0,10833.0,-5840.0,12184.0,-6936.0
19592201,14232.0,-8556.0,13610.0,-7856.0,14867.0,-9359.0
19592400,5649.0,191.0,6542.0,-779.0,4809.0,1245.0
19592800,9097.0,1278.0,9323.0,1412.0,9221.0,1551.0
19594200,11262.0,-5387.0,10468.0,-4863.0,12387.0,-6358.0
19595200,18425.0,-11390.0,17495.0,-10254.0,20034.0,-12398.0
19595300,11060.0,-5800.0,10335.0,-5510.0,11890.0,-6411.0
19595800,12432.0,-7962.0,_,_,12267.0,-7515.0
20000100,18149.0,-10095.0,18159.0,-9697.0,17792.0,-10597.0
20000200,7989.0,-70.0,7759.0,1511.0,8475.0,-1480.0
20000300,8677.0,41.0,8353.0,1315.0,9280.0,-1068.0
""" 

"""
Sample data: 
file: dataset/13528908058.edges:
contains data: target, source
19585800, 19590700
19585800, 19595200
19585800, 20000100
19590700, 19595200
19590700, 20000100
19591900, 19594200
19591900, 19595300
19591900, 19595800
19592201, 19595800
19592400, 20000200
19592800, 20000200
19592800, 20000300
19594200, 19595300
19594200, 19595800
19595200, 20000100
19595300, 19595800
20000200, 20000300
19502500, -1

corresponding nodes file: 
file: dataset/13528908058.nodes
contains data: node id, current x, current y, previous x, previous y, future x, future y
19502500,40050.0,-16544.0,40176.0,-16619.0,40205.0,-16357.0
19585800,16802.0,-11108.0,16140.0,-10573.0,17831.0,-11792.0
19590700,16846.0,-10526.0,16079.0,-9694.0,17528.0,-11131.0
19591900,11346.0,-6253.0,10833.0,-5840.0,12184.0,-6936.0
19592201,14232.0,-8556.0,13610.0,-7856.0,14867.0,-9359.0
19592400,5649.0,191.0,6542.0,-779.0,4809.0,1245.0
19592800,9097.0,1278.0,9323.0,1412.0,9221.0,1551.0
19594200,11262.0,-5387.0,10468.0,-4863.0,12387.0,-6358.0
19595200,18425.0,-11390.0,17495.0,-10254.0,20034.0,-12398.0
19595300,11060.0,-5800.0,10335.0,-5510.0,11890.0,-6411.0
19595800,12432.0,-7962.0,_,_,12267.0,-7515.0
20000100,18149.0,-10095.0,18159.0,-9697.0,17792.0,-10597.0
20000200,7989.0,-70.0,7759.0,1511.0,8475.0,-1480.0
20000300,8677.0,41.0,8353.0,1315.0,9280.0,-1068.0
""" 


dataset_path = "dataset/"

# Data loading functions
def find_all_scene_ids(dataset_dir):
    scene_ids = []
    for file in os.listdir(dataset_dir):
        if file.endswith(".edges"):
            scene_id = file.split(".")[0]
            scene_ids.append(scene_id)
    return scene_ids


def load_all_subgraphs(dataset_dir):
    scene_ids = find_all_scene_ids(dataset_dir)
    scenes = []
    for scene_id in scene_ids:
        edges_file = os.path.join(dataset_dir, f"{scene_id}.edges")
        nodes_file = os.path.join(dataset_dir, f"{scene_id}.nodes")
        if not os.path.exists(edges_file) or not os.path.exists(nodes_file):
            print(f"Skipping scene ID {scene_id}: Missing files.")
            continue

        # Load edges and nodes.
        edges = pd.read_csv(edges_file, sep=",", header=None, names=["target", "source"])
        nodes = pd.read_csv(
            nodes_file,
            sep=",",
            header=None,
            names=["node_id", "current_x", "current_y", "previous_x", "previous_y", "future_x", "future_y"],
        )
        for col in nodes.columns:
            nodes[col] = pd.to_numeric(nodes[col], errors="coerce")
        if nodes.isnull().any().any():
            nan_nodes = nodes[nodes.isnull().any(axis=1)]
            nan_node_ids = nan_nodes["node_id"].tolist()
            #print(f"Scene {scene_id}: Filtering {len(nan_node_ids)} nodes with NaN values.")
            edges = edges[~edges["source"].isin(nan_node_ids) & ~edges["target"].isin(nan_node_ids)]
            nodes = nodes.dropna(subset=["future_x", "future_y"])
        if (edges["source"] == -1).any() or (edges["target"] == -1).any():
            #print(f"Scene {scene_id} contains -1 edges. Removing these edges.")
            edges = edges[(edges["source"] != -1) & (edges["target"] != -1)]
            connected_nodes = pd.unique(edges[["target", "source"]].values.ravel())
            nodes = nodes[nodes["node_id"].isin(connected_nodes)]
        # Only append scenes with at least one node
        if len(nodes) > 0:
            scenes.append({"scene_id": scene_id, "edges": edges, "nodes": nodes})
        # else:
        #     print(f"NOTE! Scene {scene_id} skipped: no valid nodes after filtering.")
    return scenes

# scenes = load_all_subgraphs(dataset_path)
# print(f"Loaded {len(scenes)} scenes.")

# print("scenes 1:", scenes[:1])


# Function to load all scenes while keeping each scene as a separate graph
def load_all_scenes():
    scene_ids = [file.split("/")[-1].split(".")[0] for file in glob.glob(f"{dataset_path}*.nodes")]
    
    all_scenes = []
    for scene_id in scene_ids:
        edges_file = os.path.join(dataset_path, f"{scene_id}.edges")
        nodes_file = os.path.join(dataset_path, f"{scene_id}.nodes")

        edges_df = pd.read_csv(edges_file, header=None, names=["target", "source"], na_values="_")

        nodes_df = pd.read_csv(
            nodes_file,
            header=None,
            names=["node_id", "current_x", "current_y", "previous_x", "previous_y", "future_x", "future_y"],
            na_values="_"
        )


        if nodes_df.isnull().any().any():
            nan_nodes = nodes_df[nodes_df.isnull().any(axis=1)]
            nan_node_ids = nan_nodes["node_id"].tolist()
            #print(f"Scene {scene_id}: Filtering {len(nan_node_ids)} nodes with NaN values.")
            edges_df = edges_df[~edges_df["source"].isin(nan_node_ids) & ~edges_df["target"].isin(nan_node_ids)]
            nodes_df = nodes_df.dropna(subset=["future_x", "future_y"])

        if (edges_df["source"] == -1).any() or (edges_df["target"] == -1).any():
            #print(f"Scene {scene_id} contains -1 edges. Removing these edges.")
            edges_df = edges_df[(edges_df["source"] != -1) & (edges_df["target"] != -1)]
            connected_nodes = pd.unique(edges_df[["target", "source"]].values.ravel())
            nodes_df = nodes_df[nodes_df["node_id"].isin(connected_nodes)]
        
        # Convert "_" to NaN
        nodes_df = nodes_df.replace('_', np.nan)
        edges_df = edges_df.replace('_', np.nan)
        
        # Filter out nodes with missing future positions
        #nodes_df = nodes_df.dropna(subset=["future_x", "future_y"])
        
        # Create mapping for node ids within this scene
        #node_id_to_index = {node_id: idx for idx, node_id in enumerate(nodes_df["node_id"].values)}
        
        # Process edges using the node id mapping
        # edges_df = edges_df.dropna()
        # edges_df['target'] = edges_df['target'].apply(lambda x: node_id_to_index.get(x, -1))
        # edges_df['source'] = edges_df['source'].apply(lambda x: node_id_to_index.get(x, -1))
        # edges_df = edges_df[(edges_df['target'] != -1) & (edges_df['source'] != -1)]

        # Calculate motion features
        # nodes_df["dir_x"] = nodes_df["current_x"] - nodes_df["prev_x"]
        # nodes_df["dir_y"] = nodes_df["current_y"] - nodes_df["prev_y"]
        # nodes_df["speed"] = np.sqrt(nodes_df["dir_x"]**2 + nodes_df["dir_y"]**2)
        
        # Extract features and labels
        # node_features = nodes_df[["current_x", "current_y", "prev_x", "prev_y"]].values
        # labels = nodes_df[["future_x", "future_y"]].values
        # edges = edges_df[["target", "source"]].values
        
        # Store as a structured scene
        if len(nodes_df) > 0:
            all_scenes.append({"scene_id": scene_id, "edges": edges_df, "nodes": nodes_df})
    
    print(len(all_scenes))

    return all_scenes

feature_cols = ["current_x", "current_y", "previous_x", "previous_y"]
target_cols = ["future_x", "future_y"]

def convert_scene_to_tensors(scene):
    nodes_df = scene["nodes"].reset_index(drop=True)
    edges_df = scene["edges"].reset_index(drop=True)
    # Map node_id to local index
    node_id_to_idx = {nid: i for i, nid in enumerate(nodes_df["node_id"])}
    # Remap edges to local indices
    edges_df = edges_df.copy()
    edges_df["target"] = edges_df["target"].map(node_id_to_idx)
    edges_df["source"] = edges_df["source"].map(node_id_to_idx)
    # Drop edges with missing nodes (after mapping)
    edges_df = edges_df.dropna().astype(int)
    features = nodes_df[feature_cols].to_numpy().astype(np.float32)
    targets = nodes_df[target_cols].to_numpy().astype(np.float32)
    edges = edges_df.to_numpy().astype(np.int32)
    return features, edges, targets


def split_scenes(scenes, train_ratio=0.7, val_ratio=0.15):
    np.random.shuffle(scenes)
    n_total = len(scenes)
    n_train = int(n_total * train_ratio)
    n_val = int(n_total * val_ratio)
    train_scenes = scenes[:n_train]
    val_scenes = scenes[n_train : n_train + n_val]
    test_scenes = scenes[n_train + n_val :]
    return train_scenes, val_scenes, test_scenes


scenes = load_all_scenes()
print(f"Loaded {len(scenes)} scenes.")
train_scenes, val_scenes, test_scenes = split_scenes(scenes, train_ratio=0.7, val_ratio=0.15)
print(f"Train scenes: {len(train_scenes)}, Val scenes: {len(val_scenes)}, Test scenes: {len(test_scenes)}")


# """ Om man vill ha 70% Training set """
train_scenes = scenes[:int(0.7 * len(scenes))]
val_scenes = scenes[int(0.7 * len(scenes)):int(0.9 * len(scenes))]
test_scenes = scenes[int(0.9 * len(scenes)):]

# """ Om man vill ha 50% Training set """
# # train_scenes = scenes[:int(0.5 * len(scenes))]
# # val_scenes = scenes[int(0.5 * len(scenes)):int(0.9 * len(scenes))]
# # test_scenes = scenes[int(0.9 * len(scenes)):]

print(f"Training scenes: {len(train_scenes)}")
print(f"Validation scenes: {len(val_scenes)}")
print(f"Test scenes: {len(test_scenes)}")



def scene_generator(scene_list):
    for scene in scene_list:
        yield convert_scene_to_tensors(scene)


train_dataset = tf.data.Dataset.from_generator(
    lambda: scene_generator(train_scenes),
    output_signature=(
        tf.TensorSpec(shape=(None, len(feature_cols)), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.int32),
        tf.TensorSpec(shape=(None, len(target_cols)), dtype=tf.float32),
    ),
)
val_dataset = tf.data.Dataset.from_generator(
    lambda: scene_generator(val_scenes),
    output_signature=(
        tf.TensorSpec(shape=(None, len(feature_cols)), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.int32),
        tf.TensorSpec(shape=(None, len(target_cols)), dtype=tf.float32),
    ),
)
test_dataset = tf.data.Dataset.from_generator(
    lambda: scene_generator(test_scenes),
    output_signature=(
        tf.TensorSpec(shape=(None, len(feature_cols)), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2), dtype=tf.int32),
        tf.TensorSpec(shape=(None, len(target_cols)), dtype=tf.float32),
    ),
)


def squeeze_batch(features, edges, targets):
    return tf.squeeze(features, axis=0), tf.squeeze(edges, axis=0), tf.squeeze(targets, axis=0)


# train_dataset = train_dataset.shuffle(100).batch(1).map(squeeze_batch)
# val_dataset = val_dataset.batch(1).map(squeeze_batch)
# test_dataset = test_dataset.batch(1).map(squeeze_batch)

train_dataset = train_dataset.shuffle(100).batch(1).map(squeeze_batch)
val_dataset = val_dataset.batch(1).map(squeeze_batch)
test_dataset = test_dataset.batch(1).map(squeeze_batch)


189
Loaded 189 scenes.
Train scenes: 132, Val scenes: 28, Test scenes: 29
Training scenes: 132
Validation scenes: 38
Test scenes: 19


## GAT model implementation

In [17]:
class GraphAttention(layers.Layer):
    def __init__(
        self,
        units,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel",
        )
        self.kernel_attention = self.add_weight(
            shape=(self.units * 2, 1),
            trainable=True,
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel_attention",
        )
        self.built = True

    def call(self, inputs):
        node_states, edges = inputs

        # Linearly transform node states
        node_states_transformed = tf.matmul(node_states, self.kernel)

        # (1) Compute pair-wise attention scores
        node_states_expanded = tf.gather(node_states_transformed, edges)
        node_states_expanded = tf.reshape(
            node_states_expanded, (tf.shape(edges)[0], -1)
        )
        attention_scores = tf.nn.leaky_relu(
            tf.matmul(node_states_expanded, self.kernel_attention)
        )
        attention_scores = tf.squeeze(attention_scores, -1)

        # (2) Normalize attention scores
        attention_scores = tf.math.exp(tf.clip_by_value(attention_scores, -2, 2))
        attention_scores_sum = tf.math.unsorted_segment_sum(
            data=attention_scores,
            segment_ids=edges[:, 0],
            num_segments=tf.reduce_max(edges[:, 0]) + 1,
        )
        attention_scores_sum = tf.repeat(
            attention_scores_sum, tf.math.bincount(tf.cast(edges[:, 0], "int32"))
        )
        attention_scores_norm = attention_scores / attention_scores_sum
        num_nodes = tf.shape(node_states)[0]
        # (3) Gather node states of neighbors, apply attention scores and aggregate
        node_states_neighbors = tf.gather(node_states_transformed, edges[:, 1])
        out = tf.math.unsorted_segment_sum(
            data=node_states_neighbors * attention_scores_norm[:, tf.newaxis],
            segment_ids=edges[:, 0],
            num_segments=num_nodes,
        )
        return out


class MultiHeadGraphAttention(layers.Layer):
    def __init__(self, units, num_heads=8, merge_type="concat", **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.merge_type = merge_type
        self.attention_layers = [GraphAttention(units) for _ in range(num_heads)]

    def call(self, inputs):
        atom_features, pair_indices = inputs

        # Obtain outputs from each attention head
        outputs = [
            attention_layer([atom_features, pair_indices])
            for attention_layer in self.attention_layers
        ]
        # Concatenate or average the node states from each head
        if self.merge_type == "concat":
            outputs = tf.concat(outputs, axis=-1)
        else:
            outputs = tf.reduce_mean(tf.stack(outputs, axis=-1), axis=-1)
        # Activate and return node states
        return tf.nn.relu(outputs)


class GraphAttentionNetwork(keras.Model):
    def __init__(self, hidden_units, num_heads, num_layers, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.preprocess = layers.Dense(hidden_units * num_heads, activation="relu")
        self.attention_layers = [MultiHeadGraphAttention(hidden_units, num_heads) for _ in range(num_layers)]
        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs, training=False):
        # Since our dataset is unbatched, node_features is expected to have shape [n_nodes, feature_dim]
        node_features, edges = inputs

        # Directly use inputs since no batch dimension is present.
        x = self.preprocess(node_features)
        for attn_layer in self.attention_layers:
            x_new = attn_layer([x, edges])
            x = x + x_new  # residual connection
        outputs = self.output_layer(x)
        return outputs

    def train_step(self, data):
        node_features, edges, targets = data
        with tf.GradientTape() as tape:
            outputs = self([node_features, edges], training=True)
            loss = self.compiled_loss(targets, outputs)
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.compiled_metrics.update_state(targets, outputs)
        return {m.name: m.result() for m in self.metrics}


    def predict_step(self, data):
        node_features, edges, _ = data
        outputs = self([node_features, edges], training=False)
        return outputs

    def test_step(self, data):
        node_features, edges, targets = data
        outputs = self([node_features, edges], training=False)
        loss = self.compiled_loss(targets, outputs)
        self.compiled_metrics.update_state(targets, outputs)
        return {m.name: m.result() for m in self.metrics}


## Train the model

In [ ]:
# Define hyper-parameters
HIDDEN_UNITS = 100
NUM_HEADS = 8
NUM_LAYERS = 3
OUTPUT_DIM = 2


NUM_EPOCHS = 25
BATCH_SIZE = 64
LEARNING_RATE = 1e-5

loss_fn = keras.losses.MeanAbsoluteError(name="mean_absolute_error")
optimizer = keras.optimizers.Adam(
    learning_rate=LEARNING_RATE, 
    clipnorm=1.0,
    epsilon=1e-8
)
accuracy_fn = keras.metrics.MeanAbsoluteError(name="mean_absolute_error")

# Build model
gat_model = GraphAttentionNetwork(
    hidden_units=HIDDEN_UNITS, num_heads=NUM_HEADS, num_layers=NUM_LAYERS, output_dim=OUTPUT_DIM
)

# Compile model
gat_model.compile(loss=loss_fn, optimizer=optimizer, metrics=[accuracy_fn])

# Train the model
history = gat_model.fit(
    train_dataset,
    validation_data=val_dataset,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    verbose=1,
)

# Evaluate on test set
_, test_accuracy = gat_model.evaluate(x=test_dataset, verbose=1)
print(f"Test Mean Absolute Error: {test_accuracy}")

all_predictions = []
all_ground_truth = []
sample_count = 0

print("Evaluating model on entire test dataset")

# Iterate through the test dataset
for features, edges, targets in test_dataset:
    # Make predictions for this graph
    predictions = gat_model((features, edges), training=False)
    
    # Convert to numpy for easier handling
    pred_np = predictions.numpy()
    targets_np = targets.numpy()
    
    # Store predictions and ground truth
    all_predictions.append(pred_np)
    all_ground_truth.append(targets_np)
    
    # Count samples
    sample_count += len(pred_np)

print(f"Processed {len(all_predictions)} scenes with {sample_count} total nodes")

# Concatenate all predictions and ground truth values
all_pred_concat = np.concatenate(all_predictions)
all_gt_concat = np.concatenate(all_ground_truth)

# Calculate MAE for x and y coordinates separately
mae_x = np.mean(np.abs(all_pred_concat[:, 0] - all_gt_concat[:, 0]))
mae_y = np.mean(np.abs(all_pred_concat[:, 1] - all_gt_concat[:, 1]))

# Calculate overall MAE
mae = np.mean(np.abs(all_pred_concat - all_gt_concat))

# Calculate Euclidean distance for each prediction
euclidean_distances = np.sqrt(np.sum((all_pred_concat - all_gt_concat)**2, axis=1))
mean_euclidean = np.mean(euclidean_distances)
median_euclidean = np.median(euclidean_distances)

print("\n===== Evaluation Metrics =====")
print(f"Mean Absolute Error (overall): {mae:.2f}")
print(f"Mean Absolute Error (x-coordinate): {mae_x:.2f}")
print(f"Mean Absolute Error (y-coordinate): {mae_y:.2f}")
print(f"Mean Euclidean Distance: {mean_euclidean:.2f} units")
print(f"Median Euclidean Distance: {median_euclidean:.2f} units")

Epoch 1/25
    128/Unknown 26s 12ms/step - mean_absolute_error: 14391.0889 - loss: 2165.5344

2025-04-16 14:24:15.571210: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


132/132 ━━━━━━━━━━━━━━━━━━━━ 30s 37ms/step - mean_absolute_error: 14220.5947 - loss: 2199.7568 - val_loss: 4725.1616
Epoch 2/25


2025-04-16 14:24:18.819409: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


129/132 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - mean_absolute_error: 6468.7271 - loss: 4130.5596

...

2025-04-16 14:25:12.688899: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


132/132 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - mean_absolute_error: 1017.8345 - loss: 5651.7231 - val_loss: 6012.9878
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - mean_absolute_error: 820.0985 - loss: 4822.5649


2025-04-16 14:25:12.926733: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-04-16 14:25:13.069901: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


Test Mean Absolute Error: {'mean_absolute_error': <tf.Tensor: shape=(), dtype=float32, numpy=813.6096>}
Evaluating model on entire test dataset
Processed 19 scenes with 161 total nodes

===== Evaluation Metrics =====
Mean Absolute Error (overall): 813.61
Mean Absolute Error (x-coordinate): 861.34
Mean Absolute Error (y-coordinate): 765.88
Mean Euclidean Distance: 1278.27 units
Median Euclidean Distance: 1070.49 units


2025-04-16 14:25:17.455824: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
